In [32]:
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
from torch.utils.data import Dataset
import scipy.sparse as sp
from scipy import sparse
import torch
import random
import time
import bottleneck as bn

In [26]:
cfg = {
    "K": 300,
    "data_dir" : "./dataset",
    "data_file" : "user_problem_mat.npz",
    "test_batch_size" : 32,
    "test_ratio" : 0.3,
    "topks" : [10,20,50]
}

# DataSet

In [69]:
# Dataset 상속
class DATASET(Dataset): 
    def __init__(self, sparse_matrix, test_ratio=0.3, edit_mat=True, log=True, noise=False, noise_ratio=0.2):
        assert type(sparse_matrix) == np.ndarray
        
        self.sparse_matrix = sparse_matrix
        self.edit_mat = edit_mat

        self.maxK = max(cfg['topks'])
        self.test_dict = {}

        self.len_test_ones = 0
        self.len_train_ones = 0
        
        len_test_fail = 0
        nz_probs_all = self.sparse_matrix.nonzero()
        nz_probs = self.sparse_matrix.sum(0).nonzero()[0]
        
        if noise:
            len_nz_probs = len(nz_probs_all[0])
            len_noise = (int)(len_nz_probs * noise_ratio)
            noise_idxs = random.sample(range(0,len_nz_probs),len_noise)                                                          
            nz_probs_noised = (nz_probs_all[0][noise_idxs],nz_probs_all[1][noise_idxs])
            self.sparse_matrix[nz_probs_noised] = 2
        
        for user in range(len(self.sparse_matrix)):      
            items = self.sparse_matrix[user].nonzero()[0]
            len_items = len(items)
            len_sample = int(len_items * test_ratio)
            
            if len_sample >= self.maxK :
                sample_nums = random.sample(range(0,len_items),len_sample)                                                          
                sampled_items = items[sample_nums]
                                
                #sampled_items = list(set(sampled_items) - slp_idx)
                
                if edit_mat:
                    self.sparse_matrix[user][sampled_items] = 0.
                self.test_dict[user] = sampled_items  
                self.len_test_ones += len(sampled_items)
                self.len_train_ones += (len_items-len(sampled_items))
            else:
                len_test_fail += 1
        
        self.nz_probs = self.sparse_matrix.sum(0).nonzero()[0]
        if log:
            print(f"\n{len(nz_probs) - len(self.nz_probs)}")

            print("complete making test dict")
            print(f"length of failed test data : {len_test_fail}\n")
            print(f"train dict length : {self.len_train_ones}")
            print(f"test dict length : {self.len_test_ones}")
            print(f"set train_test_ratio : {test_ratio}")
            print(f"result train_test_ratio : {self.len_test_ones / (self.len_train_ones+self.len_test_ones)}")

            # 총 데이터의 개수를 리턴
    def __len__(self): 
        return len(self.sparse_matrix)

    # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
    def __getitem__(self, idx): 
        x = self.sparse_matrix[idx]
        return x
            
    # 유저가 푼 문제번호 반환
    def get_user_pos_items(self, users):
        posItems = []
        for user in users:
            posItems.append(self.sparse_matrix[user].nonzero()[0])
        return posItems

In [90]:
data_path = f'{cfg["data_dir"]}/{cfg["data_file"]}'
train_s_mat = sparse.load_npz(data_path).astype(np.float32).toarray()
train_dataset = DATASET(train_s_mat, test_ratio=cfg['test_ratio'], noise=False)


1026
complete making test dict
length of failed test data : 26

train dict length : 493564
test dict length : 209924
set train_test_ratio : 0.3
result train_test_ratio : 0.2984045214701601


In [71]:
test_s_mat = sparse.load_npz(data_path).astype(np.float32).toarray()
test_dataset = DATASET(test_s_mat, test_ratio=cfg['test_ratio'])


1040
complete making test dict
length of failed test data : 26

train dict length : 493564
test dict length : 209924
set train_test_ratio : 0.3
result train_test_ratio : 0.2984045214701601


# Utils

In [88]:
import gc

def clear():
    torch.cuda.empty_cache()
    gc.collect()

#########################################
################# Test ##################
#########################################

def getLabel(test_data, pred_data):
    r = []
    for i in range(len(test_data)):
        groundTrue = test_data[i]
        predictTopK = pred_data[i]
        pred = list(map(lambda x: x in groundTrue, predictTopK))
        pred = np.array(pred).astype("float")
        r.append(pred)
    return np.array(r).astype('float')

def NDCGatK_r(test_data,r,k):
    """
    Normalized Discounted Cumulative Gain
    rel_i = 1 or 0, so 2^{rel_i} - 1 = 1 or 0
    """
    assert len(r) == len(test_data)
    pred_data = r[:, :k]

    test_matrix = np.zeros((len(pred_data), k))
    for i, items in enumerate(test_data):
        length = k if k <= len(items) else len(items)
        test_matrix[i, :length] = 1
    max_r = test_matrix
    idcg = np.sum(max_r * 1./np.log2(np.arange(2, k + 2)), axis=1)
    dcg = pred_data*(1./np.log2(np.arange(2, k + 2)))
    dcg = np.sum(dcg, axis=1)
    idcg[idcg == 0.] = 1.
    ndcg = dcg/idcg
    ndcg[np.isnan(ndcg)] = 0.
    return np.sum(ndcg)

def RecallPrecision_ATk(test_data, r, k):
    """
    test_data should be a list? cause users may have different amount of pos items. shape (test_batch, k)
    pred_data : shape (test_batch, k) NOTE: pred_data should be pre-sorted
    k : top-k
    """
    right_pred = r[:, :k].sum(1)
    precis_n = k
    recall_n = np.array([len(test_data[i]) for i in range(len(test_data))])
    recall = np.sum(right_pred/recall_n)
    precis = np.sum(right_pred)/precis_n
    return {'recall': recall, 'precision': precis}

def test_one_batch(X, cfg):
    sorted_items = X[0].numpy()
    groundTrue = X[1]
    r = getLabel(groundTrue, sorted_items)
    pre, recall, ndcg = [], [], []
    for k in cfg['topks']:
        ret = RecallPrecision_ATk(groundTrue, r, k)
        pre.append(ret['precision'])
        recall.append(ret['recall'])
        ndcg.append(NDCGatK_r(groundTrue,r,k))
    return {'recall':np.array(recall), 
            'precision':np.array(pre), 
            'ndcg':np.array(ndcg)}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36'}    
    
def add_to_user_problem_mat(idx, id, user_problem_mat : np.array):
    data = requests.get(f'https://www.acmicpc.net/user/{id}', headers=headers)
    soup = BeautifulSoup(data.text, 'html.parser')
    trs = soup.select('div.problem-list')

    for tr in trs:
        problem_nums = tr.select('a')
            
        for problem_num in problem_nums :

            problem_num = int(problem_num.text) - 1000
            #print(problem_num)
            try:
                user_problem_mat[idx,problem_num] = 1
            except:
                print("범위를 벗어난 문제 번호 : " + str(problem_num))

def print_result(res):
    print(f'precision : {res["precision"]}')
    print(f'recall : {res["recall"]}')
    print(f'ndcg : {res["ndcg"]}')     
    
def random_sample(num_col, num_row, min_ratio=0.01, max_ratio=0.02):
    test_s_mat_rand = np.zeros((num_col,num_row))

    for i in range(num_col):
        min_num = int(num_row * min_ratio)
        max_num = int(num_row * max_ratio)
        num_sample = random.sample(range(min_num,max_num),1)[0]

        num_list = random.sample(range(num_row),num_sample)
        
        test_s_mat_rand[i][num_list] = 1
    
    return test_s_mat_rand

def print_sparcity(mat):
    if type(mat) != np.array : mat = np.array(mat)
    
    len_zeros = len(np.where(mat == 0)[0])
    len_ones = len(np.where(mat != 0)[0])
    sparcity = len_zeros / (len_zeros + len_ones)
    print(f"0 개수 : {len_zeros}")
    print(f"1 개수 : {len_ones}")
    print(f"sparsity : {sparcity}")

def minibatch(*tensors, **kwargs):

    batch_size = kwargs.get('batch_size', cfg['test_batch_size'])

    if len(tensors) == 1:
        tensor = tensors[0]
        for i in range(0, len(tensor), batch_size):
            yield tensor[i:i + batch_size]
    else:
        for i in range(0, len(tensors[0]), batch_size):
            yield tuple(x[i:i + batch_size] for x in tensors)

def test_loop(dataset, Recmodel, cfg, exclude=True, addPos=False, is_rand=False):    
    u_batch_size = cfg['test_batch_size']
    testDict: dict = dataset.test_dict
        
    max_K = max(cfg['topks'])
    results = {'precision': np.zeros(len(cfg['topks'])),
               'recall': np.zeros(len(cfg['topks'])),
               'ndcg': np.zeros(len(cfg['topks']))}
    
    users = list(testDict.keys())
    try:
        assert u_batch_size <= len(users) / 10
    except AssertionError:
        print(f"test_u_batch_size is too big for this dataset, try a small one {len(users) // 10}")
    users_list = []
    rating_list = []
    groundTrue_list = []
    # auc_record = []
    # ratings = []
    total_batch = len(users) // u_batch_size + 1
    for batch_users in minibatch(users, batch_size=u_batch_size):
        groundTrue = [list(testDict[u]) for u in batch_users]
        _batch_users = dataset.sparse_matrix[batch_users]

        if is_rand:
            rating_K = torch.Tensor([random.sample(range(dataset.sparse_matrix.shape[1]),max_K) for i in range(u_batch_size)])
        else:
            rating = Recmodel.getUsersRating(_batch_users)

            #rating = rating.cpu()
            #print('Exclude Rated Item')
            allPos = dataset.get_user_pos_items(batch_users)

            if exclude:
                exclude_index = []
                exclude_items = []
                for range_i, items in enumerate(allPos):
                    exclude_index.extend([range_i] * len(items))
                    exclude_items.extend(items)
                rating[exclude_index, exclude_items] = -(1<<10)

            if addPos:
                for i in range(len(batch_users)):
                    groundTrue[i].extend(allPos[i])

            _, rating_K = torch.topk(torch.FloatTensor(rating), k=max_K)

            del rating    
        
        users_list.append(batch_users)
        rating_list.append(rating_K.cpu())
        groundTrue_list.append(groundTrue)

    assert total_batch == len(users_list)
    X = zip(rating_list, groundTrue_list)
    pre_results = []
    for x in X:
        pre_results.append(test_one_batch(x,cfg))
    scale = float(u_batch_size/len(users))
    for result in pre_results:
        results['recall'] += result['recall']
        results['precision'] += result['precision']
        results['ndcg'] += result['ndcg']
    results['recall'] /= float(len(users))
    results['precision'] /= float(len(users))
    results['ndcg'] /= float(len(users))
    return results

# EASE

In [12]:
class EASE():
    """
    Embarrassingly Shallow Autoencoders model class
    """

    def __init__(self, lambda_):
        self.B = None
        self.lambda_ = lambda_

    def train(self, interaction_matrix):
        """
        train pass
        :param interaction_matrix: interaction_matrix
        """
        # X ~= X * B인 B구하는 것이 목표
        X = interaction_matrix
        G = X.T @ X
        diag = list(range(G.shape[0]))
        G[diag, diag] += self.lambda_
        P = np.linalg.inv(G) # P는 G의 inverse matrix

        # B = P * (X^T * X − diagMat(γ))
        self.B = P / -np.diag(P)
        min_dim = min(*self.B.shape)
        self.B[range(min_dim), range(min_dim)] = 0

    def forward(self, user_row):
        """
        forward pass
        """
        return user_row @ self.B
    
    def getUsersRating(self, user_row : np.array):
        return self.forward(user_row)

In [91]:
## EASE 훈련
start = time.time()

ease = EASE(cfg["K"])

ease.train(train_dataset.sparse_matrix)

end = time.time()

print(f"{end - start:.5f} sec")

215.03211 sec


### 기존 모델

In [92]:
print_result(test_loop(test_dataset, ease, cfg))

precision : [0.96204527 0.92791027 0.79650768]
recall : [0.12558379 0.24103551 0.50557657]
ndcg : [0.9682439  0.94235696 0.83906388]


In [93]:
print_result(test_loop(train_dataset, ease, cfg))

precision : [0.89854487 0.85394099 0.71822959]
recall : [0.11723741 0.22177614 0.45765978]
ndcg : [0.9096446  0.87465968 0.76498216]


### 값이 1인 데이터의 20%를 2로 바꿨을 때

In [74]:
print_result(test_loop(test_dataset, ease, cfg))

precision : [0.94987874 0.91014551 0.7803557 ]
recall : [0.12378721 0.23590224 0.49495839]
ndcg : [0.95917875 0.92821729 0.82428984]


In [75]:
print_result(test_loop(train_dataset, ease, cfg))

precision : [0.85339531 0.80697251 0.67582862]
recall : [0.11112284 0.20912132 0.43074015]
ndcg : [0.86815999 0.83057257 0.72293857]


### 랜덤으로 추천한 결과

In [89]:
print_result(test_loop(train_dataset, ease, cfg, is_rand=True))

precision : [0.0025869  0.00264753 0.00306386]
recall : [0.00031165 0.00063393 0.00183674]
ndcg : [0.00266419 0.00268368 0.00297329]
